In [1]:
from pyspark.sql import SparkSession
from pyspark import SQLContext
from pyspark import *
import configparser
import json
from pprint import pprint
#import rhinoscriptsyntax as rs
import os
import json_lines
import pandas as pd
import pandasql as pdsql
import hashlib
import sys
#import importlib.util
from importlib.machinery import SourceFileLoader
#couchabse packages import
from couchbase.bucket import Bucket
from couchbase.cluster import Cluster, PasswordAuthenticator
from couchbase.n1ql import N1QLQuery
import couchbase._libcouchbase as LCB
import couchbase.exceptions as E
from couchbase.user_constants import FMT_JSON
from couchbase._pyport import ulp

class ShudETLLoader:
    def __init__(self):
        self.helper = SourceFileLoader("ShudHelper", "/home/jovyan/work/shud/helper/helper.py").load_module()
        self.config = self.helper.ShudHelper.getConfig() 
        self.sparkSession = SparkSession\
                .builder\
                .appName('appname')\
                .config("spark.some.config.option", "some-value")\
                .getOrCreate()
        self.sqlContext = SQLContext(self.sparkSession)
        self.imagedirectory=self.config.get('directory', 'imagedirectory')
        self.datadirectory=self.config.get('directory', 'datadirectory')

        print(self.datadirectory)
    def loadData(self):
        files=self.helper.getJsonFiles(self.datadirectory)
        pushdata={}
        for file in files:
            df = self.sparkSession.read.json(file)
            df.createOrReplaceTempView("df")
            sqlDF = self.sparkSession.sql("SELECT * FROM df")           
            for row in sqlDF.rdd.collect():
                Enable=1
                Language='fr'
                Description='description'
                image_urls=row["image_urls"]
                images={}
                for image_url in image_urls:
                    sha_1 = hashlib.sha1()
                    sha_1.update(image_url)
                    imageID=sha_1.hexdigest()
                    imagePath=self.imagedirectory + imageID +".jpg"
                   # Image=self.helper.getImageBase64(imagePath)
                    images[imageID]=imagePath                    
                    
                productdetail={'productid': row["id"], 'createdat': row["domain"], 'updatedate': datetime.datetime.now(), 
                            'ProductDescription': {'descriptionid':row["id"], 'productName': row["title"],'Description': Description, 'Language': Language,'CreatedAt':datetime.datetime.now(), 'UpdatedAt':datetime.datetime.now()},
                            'ProductImage': {'Productid': row["id"], 'ImageID':row["id"], 'ImageLink':[images], 'CreatedAt':datetime.datetime.now(), 'UpdatedAt':datetime.datetime.now()}, 'Enable': Enable }
                
                pushdata[row["id"]]=productdetail        
        ########load data
        self.helper.InsertData(DBName='ShudDB',BucketName='product', Data=pushdata)        

ModuleNotFoundError: No module named 'couchbase'

In [8]:
ShudETLLoader()

/home/jovyan/work/shudStaging/data/


In [4]:
#import importlib.util
#'../shud.ini'

from importlib.machinery import SourceFileLoader

#helper = SourceFileLoader("ShudHelper", "/home/jovyan/work/shud/helper/helper.py").load_module()
#helper.ShudHelper()

#x=helper.ShudHelper.cleanHtml('self', '/home/jovyan/work/shud.ini')
#print(x)
#import sys

#sys.path.append('home/jovyan/work/shud/helper')
helper = SourceFileLoader("ShudHelper", "/home/jovyan/work/shud/helper/helper.py").load_module()
config = helper.ShudHelper.getConfig()
helper.ShudHelper.cleanHtml('aaa')
print(config)


In [ ]:
        
    def DBkey(self, bucket):
        cluster = Cluster('couchbase://localhost')
        cluster.authenticate(PasswordAuthenticator('DAG', 'jayceelyndsey07'))
        cb = cluster.open_bucket(bucket)
        ExistID = cb.n1ql_query(N1QLQuery('SELECT id FROM bucket'))
        print(ExistID)
        plain_text=[]
        for key in ExistID:
            cipher_suite = Fernet(key)
            plain_text.append(cipher_suite.decrypt(key))
        while True:
            key = Fernet.generate_key()
            cipher_suite = Fernet(key)
            x=str(uuid.uuid4())
            cipher_text = cipher_suite.encrypt(x)
            if (cipher_text not in plain_text) :
                break
        return cipher_text           
path = "grp_20180420_005907.942949.jl"


#df.select(df['id']).show()




sqlDFpro=sparkSession.sql("SELECT domain, title as ProductName, current_timestamp FROM df")
#sqlDF.show()
#df.rdd.map(lambda x: (x.id, x.domain, x.ProductName))


    
    
#sc = spark.sparkContext

path = "shudStaging/grp_20180420_005907.942949.jl"
df = sparkSession.read.json(path)

#df.select(df['id']).show()

df.createOrReplaceTempView("df")

sqlDF = sparkSession.sql("SELECT * FROM df")
sqlDFpro=sparkSession.sql("SELECT domain, title as ProductName, current_timestamp FROM df")
#sqlDFpro.show()
sqlDF.show()




''''
product
productid: uuid
createdat: domain
updatedate: sysdate
descriptionid: uuid
productName: title
''''
#df.show(5)
#pip install json-lines
#pip install pandasql
data = []
with open(path, 'rb') as json_data:
		for item in json_lines.reader(json_data):
			#print(item)
			data.append(item)
		json_data.close()
	#print data
df1 = pd.io.json.json_normalize(data)
df1.columns = df1.columns.map(lambda x: x.split(".")[-1])

str1="select id from df1"
#result=pdsql.sqldf(str1, locals())
#result.head(10)
#sqldf=sparkSession.sql("SELECT * FROM df").show()